# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
weather_data = pd.read_csv("weather_data.csv")
weather_data_df =pd.DataFrame(weather_data)
weather_data_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Rikitea,38,PF,1616376772,68,-23.1203,-134.9692,80.60,5.14
1,1,Kulhudhuffushi,56,MV,1616376899,68,6.6221,73.0700,82.85,3.31
2,2,Chanika,79,TZ,1616376913,98,-5.4167,38.0167,68.50,1.83
3,3,Vaini,75,TO,1616376771,79,-21.2000,-175.2000,84.20,6.91
4,4,Batemans Bay,100,AU,1616376907,93,-35.7167,150.1833,64.99,1.01


In [3]:
weather_data_df.dtypes

Unnamed: 0      int64
City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
import gmaps.datasets
gmaps.configure(api_key=g_key) # Fill in with your API key
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
#Humidity HeatMap
locations = weather_data_df[['Lat', 'Lng']]
weights = weather_data_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=300, point_radius=5))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
new_weather_df = weather_data_df.dropna()
new_weather_df.count()

Unnamed: 0    576
City          576
Cloudiness    576
Country       576
Date          576
Humidity      576
Lat           576
Lng           576
Max Temp      576
Wind Speed    576
dtype: int64

In [7]:
ideal_weather_condition_df = weather_data_df.loc[(weather_data_df['Max Temp'] > 70) & (weather_data_df['Max Temp'] < 80) & (weather_data_df['Wind Speed'] < 10) & (weather_data_df['Cloudiness'] == 0)].dropna()
ideal_weather_condition_df  

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
205,205,Aswān,0,EG,1616376966,22,24.0934,32.9070,74.25,2.37
228,228,Ambovombe,0,MG,1616376975,77,-25.1667,46.0833,73.65,4.74
241,241,Mahārāganj,0,IN,1616376981,23,27.1333,83.5667,73.90,0.72
248,248,Rehli,0,IN,1616376982,25,23.6333,79.0833,77.00,3.11
274,274,São João da Barra,0,BR,1616376905,89,-21.6403,-41.0511,75.56,5.06
278,278,Itu,0,BR,1616376990,65,-23.2642,-47.2992,77.00,5.75
347,347,Zunheboto,0,IN,1616377005,27,25.9667,94.5167,72.64,2.82
359,359,Pokhara,0,NP,1616377011,18,28.2333,83.9833,76.24,1.43
413,413,Port Lincoln,0,AU,1616376818,54,-34.7333,135.8667,77.23,7.20
444,444,Dwārka,0,IN,1616376911,85,22.2394,68.9678,74.16,3.04


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
#creating hotel dataframe
hotel_df =ideal_weather_condition_df[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"] =""
hotel_df.count()

City          19
Country       19
Lat           19
Lng           19
Hotel Name    19
dtype: int64

In [25]:
#set parameter
params={
    "radius":5000,
    "type":"lodging",
    "key":g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f'{lat},{lng}'
    #print(params)
    
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    address_name = requests.get(base_url,params=params)

    address_name = address_name.json()

    try:
        hotel_df.loc[index,"Hotel Name"] = address_name["results"][0]["name"]
    except(KeyError,IndexError):
        print("missing field/result...skipping.")
hotel_df

missing field/result...skipping.


,City,Country,Lat,Lng,Hotel Name
205,Aswān,EG,24.0934,32.9070,Nuba Nile
228,Ambovombe,MG,-25.1667,46.0833,RELAHARA
241,Mahārāganj,IN,27.1333,83.5667,Pt. Deen Dayal Hostel
248,Rehli,IN,23.6333,79.0833,Govt Sc/St Hostel Rehli
274,São João da Barra,BR,-21.6403,-41.0511,Pousada Porto De Canoas
278,Itu,BR,-23.2642,-47.2992,Hotel KK
347,Zunheboto,IN,25.9667,94.5167,Inspection Bungalow
359,Pokhara,NP,28.2333,83.9833,Butterfly Lodge
413,Port Lincoln,AU,-34.7333,135.8667,Port Lincoln Hotel
444,Dwārka,IN,22.2394,68.9678,Hotel Narayan Avezika Comfort


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
marker_layer= gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))